In [ ]:
import torch
from torch import nn
import torchvision
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import ToTensor
from pathlib import Path
import os
from PIL import Image
from matplotlib import pyplot as plt
import glob

In [ ]:
device = 'cuda'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/AIP/Train_Test_dataset.zip

In [ ]:
# Transforming data with torchvision.transforms

train_data_transforms = transforms.Compose([
        transforms.RandomResizedCrop(64, scale=(1,1)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
test_data_transforms = transforms.Compose([
        transforms.RandomResizedCrop(64, scale=(1,1)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

In [ ]:
from torchvision import datasets
train_data = datasets.ImageFolder(root='/content/classification_dataset/train',
                                  transform = train_data_transforms,
                                  target_transform = None)
test_data = datasets.ImageFolder(root='/content/classification_dataset/test',
                                  transform = test_data_transforms,
                                  target_transform = None)

In [ ]:
# Turn train and test datasets into DataLoader
from torch.utils.data import DataLoader
batch = 32
train_dataloader = DataLoader(train_data, batch, shuffle= True)
test_dataloader = DataLoader(test_data, batch, shuffle= True)

In [ ]:
## Creating Tiny VGG Model  
# torch.cuda.empty_cache()

class TinyVGG(nn.Module):
  def __init__(self, input_shape, hidden_units, output_shape):
    super().__init__()
    self.conv_1 = nn.Sequential(
        nn.Dropout(p=0.2),
        nn.Conv2d(in_channels=input_shape, 
                  out_channels=hidden_units,
                  kernel_size = 3,
                  stride=1,
                  padding=0),
        nn.MaxPool2d(kernel_size=2,
                     stride=2),
       
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=3,
                     stride=3)
    )
    self.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Dropout(p=0.5),
        nn.Linear(in_features=hidden_units*10*10,
                  out_features=output_shape)
    )

  def forward(self, x):
    x= self.conv_1(x)
    # print(x.shape)

    x= self.classifier(x)
    return x

In [ ]:
model = TinyVGG(input_shape=3, hidden_units=5, output_shape=6).to('cuda')
model

TinyVGG(
  (conv_1): Sequential(
    (0): Dropout(p=0.2, inplace=False)
    (1): Conv2d(3, 5, kernel_size=(3, 3), stride=(1, 1))
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): ReLU()
    (4): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Dropout(p=0.5, inplace=False)
    (2): Linear(in_features=500, out_features=6, bias=True)
  )
)

In [ ]:
### MODEL TRAINING ###
epochs = 1000
loss_fn = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(params=model.parameters(), lr=0.001)

model.train()

train_loss, train_acc = 0,0

for epoch in range(epochs):
  for batch, (X,y) in enumerate(train_dataloader):
    Input, label = X.to('cuda'), y.to('cuda')

    y_pred = model(Input)
    loss = loss_fn(y_pred, label)


    train_loss+=loss.item()

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    #### Calc Accuracy

    y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
    
    train_acc+= (y_pred_class==label).sum().item()/len(y_pred)
  train_loss = train_loss/len(train_dataloader)
  train_acc = train_acc / len(train_dataloader)
    
  ### MODEL TESTING AND ACC CALCULATION ####

  model.eval()
  test_loss, test_acc = 0, 0
  with torch.inference_mode():
    for batch , (X,y) in enumerate(test_dataloader):
      Input, label = X.to('cuda'), y.to('cuda')

      test_pred_logits = model(Input)

      loss = loss_fn(test_pred_logits, label)
      test_loss += loss.item()

      test_pred_labels = test_pred_logits.argmax(dim=1)
      test_acc+= ((test_pred_labels==label).sum().item()/len(test_pred_labels))

    test_loss = test_loss/ len(test_dataloader)
    test_acc = test_acc / len(test_dataloader)
    # if epoch%5 == 0:
    print(f'epoch: {epoch} Train Loss: {train_loss:.5f} Train Acc: {train_acc:.5f} Test Loss: {test_loss:.5f} Test Accuracy: {test_acc:.5f}')

epoch: 0 Train Loss: 1.89829 Train Acc: 0.21765 Test Loss: 1.76167 Test Accuracy: 0.20312
epoch: 1 Train Loss: 1.87232 Train Acc: 0.25692 Test Loss: 1.75960 Test Accuracy: 0.21094
epoch: 2 Train Loss: 1.86272 Train Acc: 0.26916 Test Loss: 1.74986 Test Accuracy: 0.26302
epoch: 3 Train Loss: 1.85762 Train Acc: 0.28274 Test Loss: 1.74945 Test Accuracy: 0.28125
epoch: 4 Train Loss: 1.85295 Train Acc: 0.28134 Test Loss: 1.74695 Test Accuracy: 0.25521
epoch: 5 Train Loss: 1.85574 Train Acc: 0.27905 Test Loss: 1.74347 Test Accuracy: 0.29167
epoch: 6 Train Loss: 1.84342 Train Acc: 0.30244 Test Loss: 1.73089 Test Accuracy: 0.29167
epoch: 7 Train Loss: 1.83255 Train Acc: 0.30456 Test Loss: 1.72748 Test Accuracy: 0.28385
epoch: 8 Train Loss: 1.83052 Train Acc: 0.30652 Test Loss: 1.72363 Test Accuracy: 0.30729
epoch: 9 Train Loss: 1.82783 Train Acc: 0.31215 Test Loss: 1.71993 Test Accuracy: 0.30990
epoch: 10 Train Loss: 1.82541 Train Acc: 0.32167 Test Loss: 1.72155 Test Accuracy: 0.30469
epoch: 11

KeyboardInterrupt: ignored

In [ ]:
#### Trying Data Augumentation for tackling Overfitting didnt works quite well for this case/dataset ####
#### following is the code for Augumentation ####


# train_transforms_aug = transforms.Compose([
#         transforms.Resize(size=(64,64)),
#         transforms.TrivialAugumentWide(num_magnitude_bins=10),
#         transforms.ToTensor(),
#         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
#     ])
# test_transforms_sim = transforms.Compose([
#         transforms.Resize(size=(64,64)),
#         transforms.ToTensor(),
#         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
#     ])

# train_data = datasets.ImageFolder(root='/content/classification_dataset/train',
#                                   transform = train_transforms_aug,
#                                   target_transform = None)
# test_data = datasets.ImageFolder(root='/content/classification_dataset/test',
#                                   transform = test_transforms_sim,
#                                   target_transform = None)


In [ ]:
##Following parts could also be used for getting Data##

# import random

# for dirpath, dirname, filenames in os.walk("/content/classification_dataset"):
#   print(f' there are {len(filenames)} image and {len(dirname)} directory In {dirpath}' )

# train_path = "/content/classification_dataset/train"
# test_path = "/content/classification_dataset/test"

# image_path_list = glob.glob('/content/classification_dataset/*/*/*.jpg')

# image_class = Path(random.choice(image_path_list)).parent.stem

# Image.open(random_image_path)

# class_names = train_data.class_to_idx

#Following code can be used for Data visualization from datasets i.e, train_data created by torchvision datasets library

# train_data[0]
# img = train_data[1][0]
# label = train_data[1][1]
# img.shape, img.dtype
# img_per = img.permute(1,2,0)
# plt.figure(figsize = (20,14))
# plt.imshow(img_per)
# plt.show()